In [1]:
import requests
import pandas as pd
from pprint import pprint
from operator import itemgetter
import json
import time
from datetime import datetime

from functools import reduce

from bs4 import BeautifulSoup

import random

# Добыча лайков

In [1]:
url = "https://www.instagram.com/p/Btndd-hhSF2/"

In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains

# настройки для запуска в фоновом режиме
# opts = Options()
# opts.set_headless()
# assert opts.headless

In [ ]:
# with open("patterns.json", "w") as f:
#     json.dump({"liked_by_user": class_pattern, "liked_by_button": 'zV_Nj'}, f)

## Это работает

In [ ]:
with open("patterns.json") as f:
    patterns = json.load(f)

In [ ]:
browser = Firefox()
browser.get(url)
browser.find_element_by_xpath("//a[@class='{}']".format(patterns['liked_by_button'])).click()

In [ ]:
frame = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div")
part_users = frame.text.split("\n")
[part_users[i] for i in range(0, len(part_users), 3)]

# Скроллинг

In [ ]:
browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", frame)  # рабочий вариант

In [ ]:
data = [i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))]
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    data.extend([i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))])
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Сбор данных постов, найденных по тэгу

Структура json
```python
["graphql"]["hashtag"]["edge_hashtag_to_media"]["page_info"]
```

In [11]:
def get_json(path, next_page=None, retries=5):
    """
    Returns json from url https://www.instagram.com/ + path
    
    path: str, part of url that follows https://www.instagram.com/
    next_page: str, link to next page similar to linked list, 
    got from 'cursor' field in response json
    
    returns: dict"""
    
    url = "https://www.instagram.com/" + path
    params = {
        "__a": 1
    }
    retries_counter = 0
    
    if next_page:
        params["max_id"] = {next_page}
    
    while retries_counter <= retries:
        try:
            r = requests.get(url, params=params)
            retries_counter = 0
            break
        except OSError as e:
            if GLOBAL_VERBOSE:
                print(e)
            retries_counter += 1

    try:
        return r.json()
    except json.decoder.JSONDecodeError as e:
        print(r)
        raise json.decoder.JSONDecodeError

In [12]:
def pages_by_tag(tag, max_pages, retries=5):
    """Instagram pages generator, yields n pages found by hash tag,
    where n is less or equal than max_pages
    
    tag: str, instagram hash tag - a string without '#' symbol
    max_pages: int, max n of pages to yield
    retries: int, number of attempts to get data from page if 
    
    yields: dict"""
    
    next_page = None
    page_n = 0
    path = f"explore/tags/{tag}/"
    retries_counter = 0
    
    while page_n <= max_pages:
        page_n += 1
        data = get_json(path, next_page)
        print(f"Got page {page_n}")
        
        try:
            page_info = data["graphql"]["hashtag"]["edge_hashtag_to_media"]["page_info"]
            retries_counter = 0
        except KeyError:
            if retries_counter <= retries:
                time.sleep(1)
                print(f"Attempt #{retries_counter} on page {page_n}...")
                retries_counter += 1
                continue
            else:
                print(f"Unable to get page data\ncursor: {next_page}")
                break
        
        # checking link to next page
        if page_info["has_next_page"]:
            next_page = page_info["end_cursor"]
        else:
            print("No more pages")
            break
        
        yield data

In [13]:
#использовать как декоратор
def parse_page(page_json, keys):
    """Parses instagram page json
    page_json: dict, json got from server response
    keys: iterable, list of fields to retrieve from page_json
    
    returns: dict"""
    
    parsed_data = {}
    page_posts = page_json["graphql"]["hashtag"]["edge_hashtag_to_media"]["edges"]
    for post in page_posts:
        raw_post = post["node"]
        parsed_post = {}
        for key in keys:
            try:
                parsed_post[key] = raw_post[key]
            except KeyError:
                try:
                    parsed_post[key] = raw_post["edge_media_to_caption"]["edges"][0]["node"][key]
                except KeyError:
                    if GLOBAL_VERBOSE:
                        print(f"Key {key} was not found")
                except IndexError as e:
                    if GLOBAL_VERBOSE:
                        print(f"Key {key} was not found:\n{e}")
        parsed_data[raw_post["shortcode"]] = parsed_post
    return parsed_data

In [14]:
def get_tag_data(tag, keys, max_pages=10):
    """Collects data from instagram posts found by tag
    
    tag: str, instagram hash tag without '#' symbol
    keys: fields from instagram response json to retrieve
    max_pages: int, max n of pages to parse
    
    returns: dict, with pairs <post_shortcode>: {<post_data>}"""
    
    tag_data = {}
    for page in pages_by_tag(tag, max_pages):
        time.sleep(2)
        page_data = parse_page(page, keys)
        tag_data.update(page_data)
    return tag_data

In [9]:
GLOBAL_VERBOSE = True
tag = "мастерклассспб"
keys = [
    "text",
    "taken_at_timestamp",
    "edge_liked_by",
    "owner",
    "is_video"
]
# stop_date = datetime.strptime("01.01.2019", "%d.%M.%Y").timestamp()

In [ ]:
tag_data = get_tag_data(tag, keys, max_pages=300)

Got page 1
Got page 2
Got page 3
Got page 4
Got page 5
Got page 6
Got page 7
Got page 8
Got page 9
Got page 10
Got page 11
Got page 12
Key text was not found:
list index out of range
Got page 13
Got page 14
Got page 15
Got page 16
Got page 17
Got page 18
Got page 19
Got page 20
Got page 21
Got page 22
Got page 23
Got page 24
Got page 25
Got page 26
Got page 27
Got page 28
Got page 29
Got page 30
Got page 31
Got page 32
Got page 33
Got page 34
Got page 35
Got page 36
Got page 37
Got page 38
Got page 39
Got page 40
Got page 41
Got page 42
Got page 43
Got page 44
Got page 45
Got page 46
Got page 47
Got page 48
Got page 49
Got page 50
Got page 51
Got page 52
Got page 53
Got page 54
Got page 55
Got page 56
Got page 57
Got page 58
Got page 59
Key text was not found:
list index out of range
Got page 60
Got page 61
Got page 62
Got page 63
Got page 64
Got page 65
Got page 66
Got page 67
Got page 68
Got page 69
Got page 70
Got page 71
Got page 72
Got page 73
Got page 74
Got page 75
Got page 76
G

In [ ]:
get_page_data(next(pages_by_tag(tag, 1)), keys)

In [81]:
print(json.dumps([{1: 2}]))

[{"1": 2}]


### Какие то тесты

In [ ]:
data = get_tag_data("мастеркласс", keys, max_pages=300)

In [ ]:
data_pd = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
data_pd.reset_index(inplace=True)

In [ ]:
data_pd.to_csv("тэг_мастеркласс_300_страниц.csv")

In [ ]:
data_pd["taken_at_timestamp"].map(lambda x: datetime.fromtimestamp(x))

In [ ]:
# html class for liked by
liked_by_class = "pbNvD fPMEg"

In [ ]:
r = requests.get("https://www.instagram.com/p/Bu1eDsZDMSx/liked_by/")
soup = BeautifulSoup(r.content, "html.parser")